<h1><center>ОТБОР ПРИЗНАКОВ</center></h1>

Вернёмся к модели «Космолайна», предсказывающей отток клиентов. Вы уже обучили её базовую версию, а также вариант модели с дополнительными признаками. Чтобы улучшить качество предсказаний модели и увеличить скорость её работы в продакшене, избавьтесь от лишних признаков, используя методы, изученные в предыдущем уроке.

В начале определите глобальные переменные. Вот шаблон того, как это можно сделать:

In [2]:
import pandas as pd
import numpy as np
import os
import psycopg
import mlflow
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, StandardScaler,
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

import catboost as cb
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder

TABLE_NAME = "clean_users_churn" # таблица с данными
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "sign_selection" # название эксперимента
RUN_NAME = "sign_selection" 
REGISTRY_MODEL_NAME = "sign_selection" # название зарегистрированной модели 
FS_ASSETS = "fs_assets" 

# experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)

Далее загрузите данные для задачи оттока из таблицы в базе данных PostgreSQL и положите их в переменную df.

In [3]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,No,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0


После того как вы выделили категориальные колонки, закодируйте их для подачи в вашу модель:

**Обучим модель**. Подготовим данные:


In [4]:
# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]


,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
7016,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,No,0
7017,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes,1
7018,Two year,Yes,Bank transfer (automatic),105.65,6844.50,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Male,0,No,No,No,0


Обучим модель:

In [5]:

# Разделим данные на две части - для обучения и для проверки качества предсказания:
X_tr, X_val, y_tr, y_val = train_test_split(df, df['target'], stratify=df['target']) 

# Тренировочная выборка
cat_features_tr = X_tr.select_dtypes(include='object')
potential_binary_features_tr = cat_features_tr.nunique() == 2

binary_cat_features_tr = cat_features_tr[potential_binary_features_tr[potential_binary_features_tr].index]
other_cat_features_tr = cat_features_tr[potential_binary_features_tr[~potential_binary_features_tr].index]
num_features_tr = X_tr.select_dtypes(['float'])

# Валидационная выборка
cat_features_val = X_val.select_dtypes(include='object')
potential_binary_features_val = cat_features_val.nunique() == 2

binary_cat_features_val = cat_features_val[potential_binary_features_val[potential_binary_features_val].index]
other_cat_features_val = cat_features_val[potential_binary_features_val[~potential_binary_features_val].index]
num_features_val = X_val.select_dtypes(['float'])

binary_cols = binary_cat_features_tr.columns.tolist()
non_binary_cat_cols = other_cat_features_tr.columns.tolist()
num_cols = num_features_tr.columns.tolist()

# Определим список трансформаций в рамках ColumnTransformer
preprocessor = ColumnTransformer( [ ('binary', OneHotEncoder(drop='if_binary'), binary_cols),
                                    ('cat', CatBoostEncoder(), non_binary_cat_cols),
                                    ('num', StandardScaler(), num_cols) ],verbose_feature_names_out=False )

# Трансформируем исходные данные с помощью созданного preprocessor
X_tr_transformed = preprocessor.fit_transform(X_tr, y_tr)
X_val_transformed = preprocessor.transform(X_val)

# Создадим модель:
model = CatBoostClassifier()

# Обучим модель:
model.fit(X_tr_transformed, y_tr)

Learning rate set to 0.020938
0:	learn: 0.6809207	total: 63.2ms	remaining: 1m 3s
1:	learn: 0.6679019	total: 66.3ms	remaining: 33.1s
2:	learn: 0.6558429	total: 69.4ms	remaining: 23.1s
3:	learn: 0.6434182	total: 72.1ms	remaining: 17.9s
4:	learn: 0.6324112	total: 74.8ms	remaining: 14.9s
5:	learn: 0.6220749	total: 77.5ms	remaining: 12.8s
6:	learn: 0.6124651	total: 80.3ms	remaining: 11.4s
7:	learn: 0.6038166	total: 83.1ms	remaining: 10.3s
8:	learn: 0.5946159	total: 85.8ms	remaining: 9.45s
9:	learn: 0.5869049	total: 88.6ms	remaining: 8.78s
10:	learn: 0.5781957	total: 91.4ms	remaining: 8.22s
11:	learn: 0.5697779	total: 94.1ms	remaining: 7.75s
12:	learn: 0.5625322	total: 96.8ms	remaining: 7.35s
13:	learn: 0.5557972	total: 99.7ms	remaining: 7.02s
14:	learn: 0.5498941	total: 102ms	remaining: 6.72s
15:	learn: 0.5434315	total: 105ms	remaining: 6.47s
16:	learn: 0.5379960	total: 108ms	remaining: 6.24s
17:	learn: 0.5327953	total: 111ms	remaining: 6.03s
18:	learn: 0.5279874	total: 113ms	remaining: 5.8

Проверим качество нашей модели:

In [6]:
# Сделаем предсказание:
y_val_pred = model.predict(X_val_transformed)

# Определим точность:
accuracy = str(accuracy_score(y_val, y_val_pred))[:4]
print(f"Точность модели: {accuracy}")

Точность модели: 0.80


Напомним, что отбор признаков важно проводить на обучающем наборе данных, чтобы исключить переобучение.
<br>Для отбора признаков используйте библиотеку mlxtend и методы Sequential Forward Selection (SFS) и Sequential Backward Selection (SBS).

---

**Задание 1**

Отберите десять лучших признаков при помощи методов SFS и SBS. Используйте такие настройки:
- параметр кросс-валидации равен 4;
- функция floating=False, то есть выключена;
- в качестве оценщика используйте алгоритм случайного леса c числом деревьев 300, а основной метрикой будет roc_auc.

Имена отобранных признаков сохраните в переменные top_sfs и top_sbs.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

estimator = RandomForestClassifier(n_estimators=300) # ваш код здесь #

sfs = SFS(estimator, k_features=10, forward=True, floating=False, scoring='roc_auc', cv=4, n_jobs=-1) # ваш код здесь #
sbs = SFS(estimator, k_features=10, forward=False, floating=False, scoring='roc_auc', cv=4, n_jobs=-1) # ваш код здесь #

sfs = sfs.fit(X_tr_transformed, y_tr) # ваш код здесь # запустите отбор признаков
sbs = sbs.fit(X_tr_transformed, y_tr) # ваш код здесь #

top_sfs = sfs.k_feature_names_ # ваш код здесь #
top_sbs = sbs.k_feature_names_ # ваш код здесь #

print('\nSequential Forward Selection (k=10)')
print('CV Score:')
print(sfs.k_score_)

print('\nSequential Backward Selection')
print('CV Score:')
print(sbs.k_score_)

Регистрируем:

In [ ]:
from mlflow.tracking import MlflowClient
# Регистрация модели с помощью MLFlow
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(model, REGISTRY_MODEL_NAME)
    model_registred_name = REGISTRY_MODEL_NAME
    mlflow.register_model("runs:/{}/model".format(run.info.run_id), model_registred_name)
    # model_version_id = mlflow.get_latest_versions(model_registred_name)[0].version
    run_id = run.info.run_id